<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


In [2]:
train_df = pd.read_pickle("starting_data_part1.pkl").\
drop(labels=["filename", "aspect"], axis = 1)

In [3]:
#Normalize: If a image only has 2 channels, it sets all 3 channels to the 
#1st channel for easier processing. This occurs in about 1% of cases. 
def normalize(image):
    if len(image.shape) == 2:
        temp = np.zeros((image.shape[0], image.shape[1], 3))
        for i in range(3):
            temp[:,:,i] = image
        image = temp
    return image

train_df['image'] = train_df['image'].apply(normalize)

In [4]:
#Transform: Reshapes the array for processing in TensorFlow. 
def transform(arr):
    new_arr = np.zeros((len(arr), 128, 128, 3))
    for i in range(len(arr)):
        new_arr[i] = arr[i]
    return new_arr

In [5]:
#Splits the training dataframe into a train and validation set, and then
#splits those into the respective feature matrices and the labels. 
train, valid = train_test_split(train_df, test_size=0.1, random_state=42,\
                                shuffle=True, stratify=train_df['class'])

train_x, train_y = np.array(train['image']), np.array(train['class'])
train_x = transform(train_x / 255)
valid_x, valid_y = np.array(valid['image']), np.array(valid['class'])
valid_x = transform(valid_x / 255)

In [6]:
# Builds a simple CNN for image recognition, consisting of convolutional layers,
# max pooling layers, and some densely connected layers at the end leading to
# a final layer of 20 neurons that help predict the category. Regularization
# is achieved by Keras kernel_regularizers during the Dense layers, although
# it's hard to get good regularzation with a very small dataset. 
model = keras.Sequential()
model.add(Conv2D(32, (3,3), input_shape = (128, 128, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_regularizer=
                keras.regularizers.l2(0.001)))
model.add(Dense(64, activation = 'relu', kernel_regularizer=
                keras.regularizers.l2(0.05)))
model.add(Dense(20, activation = 'sigmoid', kernel_regularizer=
                keras.regularizers.l2(0.05)))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 64, epochs = 25)

Epoch 1/25
1350/1350 [==============================] - 16s 12ms/step - loss: 8.2202 - acc: 0.0733
Epoch 2/25
1350/1350 [==============================] - 7s 5ms/step - loss: 6.6284 - acc: 0.1830
Epoch 3/25
1350/1350 [==============================] - 7s 5ms/step - loss: 5.4274 - acc: 0.2252
Epoch 4/25
1350/1350 [==============================] - 7s 5ms/step - loss: 4.6156 - acc: 0.2333
Epoch 5/25
1350/1350 [==============================] - 7s 5ms/step - loss: 3.8649 - acc: 0.3244
Epoch 6/25
1350/1350 [==============================] - 6s 5ms/step - loss: 3.3827 - acc: 0.4089
Epoch 7/25
1350/1350 [==============================] - 6s 5ms/step - loss: 3.0091 - acc: 0.4570
Epoch 8/25
1350/1350 [==============================] - 6s 5ms/step - loss: 2.7301 - acc: 0.5244
Epoch 9/25
1350/1350 [==============================] - 7s 5ms/step - loss: 2.4720 - acc: 0.5785
Epoch 10/25
1350/1350 [==============================] - 6s 5ms/step - loss: 2.2188 - acc: 0.6556
Epoch 11/25
1350/1350 [====

In [7]:
#Evaluates the model on the unseen validations et. 
test_loss, test_acc = model.evaluate(valid_x, valid_y)
print('Test Loss is: ', test_loss)
print('Test Accuracy is: ', test_acc)


151/151 [==============================] - 1s 4ms/step
Test Loss is:  3.34190680491
Test Accuracy is:  0.350993376694
